***GENERATED CODE FOR aggregatetimeseries PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import pyspark
import json
import numpy as np
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def getLog(self, df):
    dfLog = np.log(df)
    return dfLog


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        df = getLog(df, )
        return df


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
#%run aggregatetimeseriesHooks.ipynb
try:
	#sourcePreExecutionHook()

	airpassengers = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/demo/AirPassengers.csv', 'filename': 'AirPassengers.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(airpassengers)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run aggregatetimeseriesHooks.ipynb
try:
	aggregation = AggregationOperation.run(["6006afba73929e1a11f23121"],{"6006afba73929e1a11f23121": airpassengers}, "6006afc073929e1a11f23122", spark, "{'dataType': 'numeric', 'dataTypes': ['String', 'numeric'], 'url': '/FileStore/platform/demo/AirPassengers.csv', 'SelectedColumns': [{'columnName': '#Passengers', 'type': 'numeric', 'disable': False}], 'functionData': {'functionName': 'aggregate', 'applyOn': [{'columnName': '#Passengers', 'type': 'numeric', 'disable': False}], 'functionsData': [{'aggregateFunction': 'count', 'aggregateOn': [{'columnName': '#Passengers', 'type': 'numeric', 'disable': False}], 'groupOn': [{'columnName': '#Passengers', 'type': 'numeric', 'disable': False}]}]}, 'allColumns': [{'columnName': 'Month', 'type': 'String', 'disable': True}, {'columnName': '#Passengers', 'type': 'numeric', 'disable': False}]}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run aggregatetimeseriesHooks.ipynb
try:
	#transformationPreExecutionHook()

	timeseriesfe = TransformationTimeSeriesForecastingMain.run(aggregation, json.dumps( {"FE": {"functionList": [{"function": "log"}, {"function": "Original"}], "featureList": [{"transformationsData": [{"feature_label": "Month", "transformation_label": "novalue"}], "feature": "Month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "15", "mean": "", "stddev": "", "min": "1949-07", "max": "1958-08", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "#Passengers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "263.07", "stddev": "135.67", "min": "104", "max": "505", "missing": "0"}, "updatedLabel": "#Passengers"}], "originalfile": "/FileStore/platform/demo/AirPassengers.csv", "features": {"timecolumn": "Month", "tocompare": "#Passengers"}, "dataPercentage": "10", "statFunction": {"function": "log", "parameter": ""}}}))

	#transformationPostExecutionHook(timeseriesfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run aggregatetimeseriesHooks.ipynb
try:
	#mlPreExecutionHook()

	modelData = driverArima(timeseriesfe)

	#mlPostExecutionHook(model)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
try:
    model=modelData['model']
    X_label=modelData['X_label']
    y_label=modelData['y_label']
    plt.clf()
    model.plot_predict(1,264 )
    plt.title('Trending Graphs')
    plt.xlabel(X_label )
    plt.ylabel(y_label )
    plt.grid(color='lightgray')
    display()
except Exception as ex:
    logging.error(ex)

